# 작업 1유형

## 결측치가 하나라도 존재하는 행의 경우 경우 해당 행을 삭제하라. 그후 남은 데이터의 상위 70%에 해당하는 데이터만 남겨둔 후 median_income 컬럼의 1분위수를 반올림하여 소숫점이하 2째자리까지 구하여라

In [7]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p1_1.csv')
df = df.dropna().reset_index(drop = True)
df = df.iloc[:int(len(df) * 0.7), :]
res = round(df['median_income'].quantile(0.25), 2)
print(res)

2.51


## 1990년도는 해당년도 평균 이하 GDP를 가지지만, 2010년도에는 해당년도 평균 이상 GDP를 가지는 국가의 숫자를 구하여라

In [19]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p1_2.csv')
df_90 = df[df['Year'] == 1990]
df_90 = df_90[df_90['Value'] <= df_90['Value'].mean()].reset_index(drop = True)['Country Name'].to_frame()

df_10 = df[df['Year'] == 2010]
df_10 = df_10[df_10['Value'] >= df_10['Value'].mean()].reset_index(drop = True)['Country Name'].to_frame()

res_df = pd.merge(df_90, df_10, on = 'Country Name', how = 'inner')
res = len(res_df.value_counts())
print(res)

5


## 데이터에서 결측치가 가장 많은 컬럼을 출력하라

In [21]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p1_3.csv')
res = df.isnull().sum().sort_values(ascending = False).index[0]

Fare


# 작업 2유형

종속 변수 : TravelInsurance , TravelInsurance가 1일 확률을 구해서 제출하라. 평가지표 : auc
제출 파일의 컬럼은 ID, proba 두개만 존재해야한다.

In [ ]:
import pandas as pd
train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p2_train_.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p2_test_.csv')
train.head()

,ID,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,1008,26,Private Sector/Self Employed,Yes,1400000,5,0,No,Yes,1
1,199,30,Private Sector/Self Employed,No,1450000,5,0,Yes,Yes,1
2,86,32,Government Sector,Yes,900000,4,0,No,No,0
3,560,26,Private Sector/Self Employed,Yes,1400000,7,0,No,Yes,1
4,161,34,Private Sector/Self Employed,No,1400000,3,1,No,Yes,1


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   1490 non-null   int64 
 1   Age                  1490 non-null   int64 
 2   Employment Type      1490 non-null   object
 3   GraduateOrNot        1490 non-null   object
 4   AnnualIncome         1490 non-null   int64 
 5   FamilyMembers        1490 non-null   int64 
 6   ChronicDiseases      1490 non-null   int64 
 7   FrequentFlyer        1490 non-null   object
 8   EverTravelledAbroad  1490 non-null   object
 9   TravelInsurance      1490 non-null   int64 
dtypes: int64(6), object(4)
memory usage: 116.5+ KB


## 방법 1. ohehotencoder

In [ ]:
train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p2_train_.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p2_test_.csv')

col_drop = ['ID']
col_num = ['Age', 'AnnualIncome', 'FamilyMembers', 'ChronicDiseases']
col_cat = ['Employment Type', 'GraduateOrNot', 'FrequentFlyer', 'EverTravelledAbroad']
col_y = ['TravelInsurance']
test_id = test['ID']

train = train.drop(col_drop, axis = 1)
test = test.drop(col_drop, axis = 1)


x_train = train.loc[:, 'Age' : 'EverTravelledAbroad']
y_train = train.loc[:, 'TravelInsurance']
x_test = test

# 명목형
x = pd.concat([x_train, x_test])
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown = 'ignore')
ohe.fit(x[col_cat])

x_train_res = ohe.transform(x_train[col_cat])
x_test_res = ohe.transform(x_test[col_cat])

x_train_ohe = pd.DataFrame(x_train_res.todense(), columns = ohe.get_feature_names_out())
x_test_ohe = pd.DataFrame(x_test_res.todense(), columns = ohe.get_feature_names_out())

x_train_fin = pd.concat([x_train_ohe, x_train[col_num]], axis = 1)
x_test_fin = pd.concat([x_test_ohe, x_test[col_num]], axis = 1)

from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(x_train_fin, y_train.values.ravel(),
                                            test_size = 0.3, stratify = y_train.values.ravel())

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(x_tr[col_num])

x_tr[col_num] = scaler.transform(x_tr[col_num])
x_val[col_num] = scaler.transform(x_val[col_num])
x_test_fin[col_num] = scaler.transform(x_test_fin[col_num])

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(x_tr, y_tr)

from sklearn.metrics import accuracy_score
y_pred_train = model.predict(x_tr)
print('accuracy tr', accuracy_score(y_pred_train, y_tr))

y_pred_val = model.predict(x_val)
print('accuracy val', accuracy_score(y_pred_val, y_val))


from sklearn.metrics import roc_auc_score
y_pred_train = model.predict_proba(x_tr)[:, 1]
print('roc_auc_score tr', roc_auc_score(y_tr, y_pred_train))

y_pred_val = model.predict_proba(x_val)[:, 1]
print('roc_auc_score val', roc_auc_score(y_val, y_pred_val))

y_pred = model.predict_proba(x_test_fin)[:, 1]
result = pd.DataFrame({'ID' : test_id, 'proba' : y_pred})
result.to_csv('00300.csv', index = False)

accuracy tr 0.9434324065196549
accuracy val 0.738255033557047
roc_auc_score tr 0.9883619377273528
roc_auc_score val 0.7661818658280923


## 방법 2. dummies

In [ ]:
train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p2_train_.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p2_test_.csv')

col_drop = ['ID']
col_num = ['Age', 'AnnualIncome', 'FamilyMembers', 'ChronicDiseases']
col_cat = ['Employment Type', 'GraduateOrNot', 'FrequentFlyer', 'EverTravelledAbroad']
col_y = ['TravelInsurance']
test_id = test['ID']

train = train.drop(col_drop, axis = 1)
test = test.drop(col_drop, axis = 1)


x_train = train.loc[:, 'Age' : 'EverTravelledAbroad']
y_train = train.loc[:, 'TravelInsurance']
x_test = test

# 명목형
x_train_dum = pd.get_dummies(x_train)
x_test_dum = pd.get_dummies(x_test)
x_test_dum = x_test_dum[x_train_dum.columns]

from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(x_train_dum, y_train.values.ravel(),
                                            test_size = 0.3, stratify = y_train.values.ravel())

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(x_tr[col_num])

x_tr[col_num] = scaler.transform(x_tr[col_num])
x_val[col_num] = scaler.transform(x_val[col_num])
x_test_dum[col_num] = scaler.transform(x_test_dum[col_num])

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(x_tr, y_tr)

from sklearn.metrics import accuracy_score
y_pred_train = model.predict(x_tr)
print('accuracy tr', accuracy_score(y_pred_train, y_tr))

y_pred_val = model.predict(x_val)
print('accuracy val', accuracy_score(y_pred_val, y_val))


from sklearn.metrics import roc_auc_score
y_pred_train = model.predict_proba(x_tr)[:, 1]
print('roc_auc_score tr', roc_auc_score(y_tr, y_pred_train))

y_pred_val = model.predict_proba(x_val)[:, 1]
print('roc_auc_score val', roc_auc_score(y_val, y_pred_val))

y_pred = model.predict_proba(x_test_dum)[:, 1]
result = pd.DataFrame({'ID' : test_id, 'proba' : y_pred})
result.to_csv('00300.csv', index = False)

accuracy tr 0.9367209971236817
accuracy val 0.7628635346756152
roc_auc_score tr 0.9889127926541993
roc_auc_score val 0.7669898672257163


# 작업 3유형

##

투약 후 체중에서 투약 전 체중을 뺏을 때 값은 일반 적으로 세가지 등급으로 나눈다. -3이하 : A등급, -3초과 0이하 : B등급, 0 초과 : C등급. 약 실험에서 A,B,C 그룹간의 인원 수 비율은 2:1:1로 알려져 있다. 위 데이터 표본은 각 범주의 비율에 적합한지 카이제곱 검정하려한다.



In [22]:
import pandas as pd
df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p3_1.csv')
df.head()


,ID,투약전,투약후
0,user_1,55.137,56.106
1,user_2,66.584,60.409
2,user_3,52.259,50.639
3,user_4,77.081,69.164
4,user_5,62.677,57.622


### A등급에 해당하는 유저는 몇명인지 확인하라

In [23]:
def label(x):
    if x <= -3:
        return 'A'
    elif -3 < x <= 0:
        return 'B'
    elif x > 0:
        return 'C'

df['diff'] = df['투약후'] - df['투약전']
df['diff_label'] = df['diff'].apply(label)

res = len(df[df['diff_label'] == 'A'])
print(res)

121


### 카이제곱검정 통계량을 반올림하여 소숫점 이하 3째자리까지 구하여라

In [34]:
from scipy.stats import chisquare

data = df.groupby('diff_label').size().sort_index().to_frame()
data['e'] = [data[0].sum()*0.5, data[0].sum()*0.25, data[0].sum()*0.25]

t, p = chisquare(data[0], data['e'])
res = round(t, 3)
print(res)

3.613


### 카이제곱 검정 p값을 반올림하여 소숫점 이하 3자리까지 구하고, 유의수준 0.05하에서 귀무가설과 대립가설중 유의한 가설을 하나를 선택하시오(귀무/대립)

In [35]:
from scipy.stats import chisquare

data = df.groupby('diff_label').size().sort_index().to_frame()
data['e'] = [data[0].sum()*0.5, data[0].sum()*0.25, data[0].sum()*0.25]

t, p = chisquare(data[0], data['e'])
res = round(p, 3)
print(res)

if res < 0.05:
    print('대립')
else:
    print('귀무')

0.164
귀무


##

A,B 공장에서 생산한 기계들의 rpm 값들을 기록한 데이터이다. 대응 표본 t 검정을 통해 B공장 제품들이 A 공장 제품들보다 rpm이 높다고 말할 수 있는지 검정하려한다

In [36]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p3_2_.csv')
df.head()


,rpm,group
0,163.6,A
1,246.9,A
2,234.9,A
3,156.9,A
4,226.6,A


### A,B 공장 각각 정규성을 가지는지 샤피로 검정을 통해 확인하라. (각 공장의 pvalue 출력할 것)



In [39]:
from scipy.stats import shapiro
a = df[df['group'] == 'A']['rpm']
b = df[df['group'] == 'B']['rpm']

t_a, p_a = shapiro(a)
t_b, p_b = shapiro(b)

if p_a < 0.05:
    print('a 정규성X')
else:
    print('a 정규성')

if p_b < 0.05:
    print('b 정규성X')
else:
    print('b 정규성')

a 정규성
b 정규성


### A,B 공장 생산 제품의 rpm은 각각 등분산성

In [41]:
from scipy.stats import levene
a = df[df['group'] == 'A']['rpm']
b = df[df['group'] == 'B']['rpm']

t, p = levene(a, b)

if p < 0.05:
    print('등분산X')
else:
    print('등분산')


등분산


### 대응 표본 t 검정을 통해 B공장 제품들의 rpm이 A 공장 제품의 rpm보다 크다고 말할 수 있는지 검정하라. pvalue를 소숫점 이하 3자리까지 출력하고 귀무가설, 대립가설 중 하나를 출력하라*

In [43]:
from scipy.stats import ttest_rel

t, p = ttest_rel(a, b, alternative = 'less')
res = round(p, 3)
print(res)

if res < 0.05:
    print('대립')
else:
    print('귀무')

0.009
대립
